<a href="https://colab.research.google.com/github/Hsuan7/homework/blob/main/w13_%E5%9F%BA%E5%9B%A0%E6%BC%94%E7%AE%97%E6%B3%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
!pip install yfinance

## 從多支股票中挑選出最優組合

In [49]:
import yfinance as yf
import numpy as np
import random

# 固定隨機種子
np.random.seed(42)
random.seed(42)

# 基因演算法參數
POPULATION_SIZE = 20  # 每代族群中的染色體數量
NUM_GENERATIONS = 200  # 總共的演化世代數
MUTATION_RATE = 0.2  # 突變的機率

# 股票代號與日期範圍
stock_symbols = ['2330', '1201', '2454', '2357', '4426', '8271']
start_date = "2020-01-01"
end_date = "2023-01-01"

In [50]:
# 下載股票數據
print("下載股票數據...")
def download_stock_data(symbols, start_date, end_date):
    data = {}
    for symbol in symbols:
        try:
            stock = yf.download(symbol + '.TW', start=start_date, end=end_date)['Adj Close']
            if not stock.empty:
                data[symbol] = stock.pct_change().dropna()  # 計算每日回報率
            else:
                print(f"警告: 無法下載 {symbol} 的數據")
        except Exception as e:
            print(f"錯誤: 無法下載 {symbol} 的數據: {e}")
    return data

stock_data = download_stock_data(stock_symbols, start_date, end_date)
if not stock_data:
    raise ValueError("無法獲取任何股票數據，請檢查代號或網絡連接！")

# 計算年化回報率與波動率
stock_returns = []
stock_risks = []

for symbol, returns in stock_data.items():
    annual_return = returns.mean() * 252  # 年化報酬率 (252 為交易日數)
    annual_risk = returns.std() * np.sqrt(252)  # 年化波動率
    stock_returns.append(annual_return)
    stock_risks.append(annual_risk)

stock_returns = np.array(stock_returns)
stock_risks = np.array(stock_risks)
num_stocks = len(stock_symbols)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

下載股票數據...



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [51]:
# 適應函數
def fitness(chromosome):
    """計算染色體的適應度值"""
    portfolio_return = np.sum(chromosome * stock_returns)
    portfolio_risk = np.sqrt(np.sum((chromosome * stock_risks) ** 2))
    stock_count = np.sum(chromosome)

    if portfolio_risk > 0:
        sharpe_ratio = portfolio_return / portfolio_risk
    else:
        sharpe_ratio = 0

    return 0.8 * sharpe_ratio - 0.006 * stock_count

# 初始化族群
def initialize_population():
    """生成隨機初始族群"""
    return [np.random.randint(0, 2, num_stocks) for _ in range(POPULATION_SIZE)]

# 選擇函數
def select(population, fitness_scores):
    """加權隨機選擇父母染色體"""
    min_fitness = min(fitness_scores)
    shifted_fitness = [f - min_fitness + 1e-6 for f in fitness_scores]
    total_fitness = sum(shifted_fitness)
    probs = [f / total_fitness for f in shifted_fitness]
    return population[np.random.choice(range(POPULATION_SIZE), p=probs)]

# 交配函數
def crossover(parent1, parent2):
    """執行單點交叉生成子代"""
    point = np.random.randint(1, max(2, num_stocks - 1))
    child1 = np.concatenate((parent1[:point], parent2[point:]))
    child2 = np.concatenate((parent2[:point], parent1[point:]))
    return child1, child2

# 突變函數
def mutate(chromosome):
    """對染色體執行突變操作"""
    for i in range(num_stocks):
        if random.random() < MUTATION_RATE:
            chromosome[i] = 1 - chromosome[i]
    return chromosome

# 主基因演算法
def genetic_algorithm():
    """基因演算法實現最佳投資組合選擇"""
    population = initialize_population()

    for generation in range(NUM_GENERATIONS):
        fitness_scores = [fitness(chromosome) for chromosome in population]
        new_population = []

        for _ in range(POPULATION_SIZE // 2):
            parent1 = select(population, fitness_scores)
            parent2 = select(population, fitness_scores)
            child1, child2 = crossover(parent1, parent2)
            new_population.append(mutate(child1))
            new_population.append(mutate(child2))

        population = new_population
        best_fitness = max(fitness_scores)
        print(f"Generation {generation + 1}: Best Fitness = {best_fitness:.4f}")

    best_index = np.argmax([fitness(chromosome) for chromosome in population])
    return population[best_index]

# 執行演算法
best_portfolio = genetic_algorithm()

print("\n最佳選股組合:", best_portfolio)
print("總報酬:", np.sum(best_portfolio * stock_returns))
print("總風險:", np.sqrt(np.sum((best_portfolio * stock_risks) ** 2)))


Generation 1: Best Fitness = 1.3727
Generation 2: Best Fitness = 1.3727
Generation 3: Best Fitness = 1.3727
Generation 4: Best Fitness = 1.3727
Generation 5: Best Fitness = 1.3727
Generation 6: Best Fitness = 1.3727
Generation 7: Best Fitness = 1.3727
Generation 8: Best Fitness = 1.3727
Generation 9: Best Fitness = 1.3727
Generation 10: Best Fitness = 1.2559
Generation 11: Best Fitness = 1.2559
Generation 12: Best Fitness = 1.3727
Generation 13: Best Fitness = 1.3727
Generation 14: Best Fitness = 1.3727
Generation 15: Best Fitness = 1.2559
Generation 16: Best Fitness = 1.2559
Generation 17: Best Fitness = 1.3727
Generation 18: Best Fitness = 1.3727
Generation 19: Best Fitness = 1.2559
Generation 20: Best Fitness = 1.3727
Generation 21: Best Fitness = 1.2559
Generation 22: Best Fitness = 1.3727
Generation 23: Best Fitness = 1.3727
Generation 24: Best Fitness = 1.3727
Generation 25: Best Fitness = 1.3727
Generation 26: Best Fitness = 1.3727
Generation 27: Best Fitness = 1.3727
Generation

In [53]:
def summarize_results(best_portfolio):
    print("\n最佳選股組合:", best_portfolio)
    print("總報酬:", np.sum(best_portfolio * stock_returns))
    print("總風險:", np.sqrt(np.sum((best_portfolio * stock_risks) ** 2)))
    print("股票對應的代碼與選擇狀態:")
    for i, symbol in enumerate(stock_symbols):
        print(f"{symbol}: {'選擇' if best_portfolio[i] == 1 else '不選擇'}")

In [54]:
summarize_results(best_portfolio)


最佳選股組合: [1 1 0 1 1 1]
總報酬: 2.9626918558863555
總風險: 1.843135587430137
股票對應的代碼與選擇狀態:
2330: 選擇
1201: 選擇
2454: 不選擇
2357: 選擇
4426: 選擇
8271: 選擇
